# Creating a parser function to manipulate gpx files

Our objecive is to create a function that can be fed *gpx* files and will compose a dataframe containing the most useful information of said route, as well as changing the name of the *gpx* file to one more easily accessed.

In [1]:
#Importing our libraries.

import pandas as pd
import time
import re
import pathlib
import os
import gpxpy
import gpxpy.gpx
import time
from pathlib import Path
import os

In [ ]:
#Defining a function to parse our gpx files while retaining the valuable data.

def parser(file):
    try:
        gpx_file = open(file, 'r') 
        gpx = gpxpy.parse(gpx_file) 
        coords = []
        alt = []
        for track in gpx.tracks:
            for segment in track.segments:        
                for point in segment.points:
                    coords.append(tuple([point.latitude, point.longitude]))
                    alt.append(point.elevation)
        parsed_file = {'name': track.name, 
                       'coords': [coords], 
                       'alt': [alt], 
                       'distance': track.length_3d()/1000, 
                       'climb': int(gpx.get_uphill_downhill()[0]),
                       'min_alt': int(gpx.get_elevation_extremes()[0]),
                       'max_alt': int(gpx.get_elevation_extremes()[1]),
                       's_la': coords[0][0],
                       's_lo': coords[0][1],
                       'f_la': coords[-1][0],
                       'f_lo': coords[-1][1],}
        df = pd.DataFrame(parsed_file)
        return df
    except:
        pass

In [ ]:
def gpx2df(gpx):    
    data = gpx.tracks[0].segments[0].points

    ## Start Position
    start = data[0]
    ## End Position
    finish = data[-1]

    df = pd.DataFrame(columns=['lon', 'lat', 'alt', 'time'])
    for point in data:
        df = df.append({'lon': point.longitude, 'lat' : point.latitude, 'alt' : point.elevation, 'time' : point.time}, ignore_index=True)
    df['time'] = df['time'].astype(str).str[:-6]
    df['time'] = pd.to_datetime(df['time'], dayfirst=True)
    return data, df

In [2]:
gpx_file = open('file.gpx', 'r')
gpx = gpxpy.parse(gpx_file)

In [3]:
data = gpx.tracks[0].segments[0].points

In [5]:
data[0]

GPXTrackPoint(42.204427421093, 2.49827971681952, elevation=377.9, time=datetime.datetime(2011, 9, 17, 21, 46, 16, tzinfo=SimpleTZ("Z")))

In [32]:
start = [segment.points[0].latitude,segment.points[0].longitude]
end = [segment.points[-1].latitude,segment.points[-1].longitude]

In [8]:
help(gpx)

Help on GPX in module gpxpy.gpx object:

class GPX(builtins.object)
 |  GPX() -> None
 |  
 |  Methods defined here:
 |  
 |  __init__(self) -> None
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __repr__(self) -> str
 |      Return repr(self).
 |  
 |  add_elevation(self, delta: float) -> None
 |      Adjusts elevation data of GPX data.
 |      
 |      Parameters
 |      ----------
 |      delta : float
 |          Elevation delta in meters to apply to GPX data
 |  
 |  add_missing_data(self, get_data_function: Callable[[gpxpy.gpx.GPXTrackPoint], Any], add_missing_function: Callable[[List[gpxpy.gpx.GPXTrackPoint], gpxpy.gpx.GPXTrackPoint, gpxpy.gpx.GPXTrackPoint, List[float]], NoneType]) -> None
 |  
 |  add_missing_elevations(self) -> None
 |  
 |  add_missing_speeds(self) -> None
 |      The missing speeds are added to a segment.
 |      
 |      The weighted harmonic mean is used to approximate the speed at
 |      a :obj:'~.GPXTrackPoint'.
 |     

In [16]:
gpx.get_points_data()[-1]

PointData(point=GPXTrackPoint(42.1084713097662, 3.14332785084844, elevation=9.8, time=datetime.datetime(2011, 9, 17, 23, 58, 37, 618000, tzinfo=SimpleTZ("Z"))), distance_from_start=84873.64539252176, track_no=0, segment_no=0, point_no=696)